In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data = load_diabetes()
x , y = data.data , data.target

print("=== DATA SUMMARY ===")
print(f"Features: {x.shape}, Target: {y.shape}")
print(f"Target range: [{y.min():.1f}, {y.max():.1f}]")

# Standardize features ONLY
scaler_x = StandardScaler()
x_scaled = scaler_x.fit_transform(x)

=== DATA SUMMARY ===
Features: (442, 10), Target: (442,)
Target range: [25.0, 346.0]


## Asumptions of Linear Regression
- 

## Linear Regression from Scikit Learn

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error , r2_score

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

lr = LinearRegression()
lr.fit(x_train,y_train)

y_pred = lr.predict(x_test)
mse_sklearn = mean_squared_error(y_test,y_pred)
r2_sklearn = r2_score(y_test,y_pred)

print(f"\nScikit-learn Results:")
print(f"MSE: {mse_sklearn:.2f}")
print(f"R²:  {r2_sklearn:.4f}")


Scikit-learn Results:
MSE: 2900.19
R²:  0.4526


## Linear Regression from scratch

## Linear Regression with pytorch

In [4]:
class LR(nn.Module):
    def __init__(self,input_dim=1,output_dim=1):
        super(LR,self).__init__()
        self.linear = nn.Linear(input_dim,output_dim)

    def forward(self,x):
        return self.linear(x)

In [5]:
x = torch.tensor(x_scaled, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).view(-1, 1)

print('\nTensor shapes:')
print('x_tensor', x.shape)  
print('y_tensor', y.shape)

x_train , x_test , y_train , y_test = train_test_split(x,y,test_size=0.2,random_state=42)
x_train_sub , x_val_sub , y_train_sub , y_val_sub = train_test_split(x_train,y_train,test_size=0.2,random_state=42)


print('\nSplit shapes:')
print('x_train_sub', x_train_sub.shape, 'y_train_sub', y_train_sub.shape)
print('x_val_sub', x_val_sub.shape, 'y_val_sub', y_val_sub.shape)
print('x_test', x_test.shape, 'y_test', y_test.shape)


Tensor shapes:
x_tensor torch.Size([442, 10])
y_tensor torch.Size([442, 1])

Split shapes:
x_train_sub torch.Size([282, 10]) y_train_sub torch.Size([282, 1])
x_val_sub torch.Size([71, 10]) y_val_sub torch.Size([71, 1])
x_test torch.Size([89, 10]) y_test torch.Size([89, 1])


In [6]:
input_dim = x.shape[1]  # 10 features for diabetes dataset
model = LR(input_dim=input_dim, output_dim=1)
loss = nn.MSELoss()
optimizer = optim.Adam(model.parameters(),lr=0.01,weight_decay=1e-5)
print(model)

LR(
  (linear): Linear(in_features=10, out_features=1, bias=True)
)


In [7]:
epochs = 100000
train_losses = []
val_losses = []
best_val_loss = float('inf')
patience = 50  # Wait 50 epochs without improvement
no_improvement_count = 0
min_delta = 10 

print(f"\n=== MODEL CONFIGURATION ===")
print(f"Initial loss: {loss(model(x_train), y_train).item():.6f}")


print("Training Linear Regression Pytorch Model...")
print(f"{'Epoch':<10} {'Train Loss':<12} {'Test Loss':<12}")
print("-" * 40)


for i , epoch in enumerate(range(epochs)):
    # Training phase
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    y_pred_train = model(x_train_sub)
    train_loss = loss(y_pred_train, y_train_sub)

    train_loss.backward()
    optimizer.step()


    model.eval()
    with torch.no_grad():
        y_pred_val = model(x_val_sub)
        val_loss = loss(y_pred_val, y_val_sub)
    
    train_losses.append(train_loss.item())
    val_losses.append(val_loss.item())


    current_val_loss = val_loss.item()
    if current_val_loss < best_val_loss - min_delta:
        best_val_loss = current_val_loss
        no_improvement_count = 0
        best_epoch = epoch
        best_weights = model.linear.weight.data.clone()
        best_bias = model.linear.bias.data.clone()
    else:
        no_improvement_count += 1

    if len(val_losses) >= 2 and no_improvement_count >= patience:
        print(f"\n EARLY STOPPING at epoch {epoch+1}")
        print(f"   No improvement for {patience} consecutive epochs")
        print(f"   Best validation loss: {best_val_loss:.2f} at epoch {best_epoch+1}")
        
        # Restore best weights
        model.linear.weight.data = best_weights
        model.linear.bias.data = best_bias
        break

    if (epoch + 1) % 500 == 0:
        print(f'Epoch {(i+1)//500}: [{epoch+1}/{epochs}], Loss: {train_loss.item():.6f}')


=== MODEL CONFIGURATION ===
Initial loss: 29723.187500
Training Linear Regression Pytorch Model...
Epoch      Train Loss   Test Loss   
----------------------------------------
Epoch 1: [500/100000], Loss: 25190.794922
Epoch 2: [1000/100000], Loss: 22748.585938
Epoch 3: [1500/100000], Loss: 20877.912109
Epoch 4: [2000/100000], Loss: 19366.705078
Epoch 5: [2500/100000], Loss: 18075.345703
Epoch 6: [3000/100000], Loss: 16909.433594
Epoch 7: [3500/100000], Loss: 15817.588867
Epoch 8: [4000/100000], Loss: 14780.897461
Epoch 9: [4500/100000], Loss: 13794.627930
Epoch 10: [5000/100000], Loss: 12857.182617
Epoch 11: [5500/100000], Loss: 11967.312500
Epoch 12: [6000/100000], Loss: 11123.917969
Epoch 13: [6500/100000], Loss: 10326.171875
Epoch 14: [7000/100000], Loss: 9573.455078
Epoch 15: [7500/100000], Loss: 8865.286133
Epoch 16: [8000/100000], Loss: 8201.208984
Epoch 17: [8500/100000], Loss: 7580.759766
Epoch 18: [9000/100000], Loss: 7003.506348
Epoch 19: [9500/100000], Loss: 6468.995117
Ep

In [8]:
model.eval()
with torch.no_grad():
   y_pred_test = model(x_test)
   test_loss = loss(y_pred_test, y_test)

   weights = model.linear.weight.squeeze()
   bias = model.linear.bias.item()

   y_test_flat = y_test.view(-1)
   y_pred_flat = y_pred_test.view(-1)
   
   ss_total = torch.sum((y_test_flat - torch.mean(y_test_flat))**2)
   ss_residual = torch.sum((y_test_flat - y_pred_flat)**2)
   r_squared = 1 - (ss_residual / ss_total)


print(f"\n{'='*50}")
print("PYTORCH RESULTS")
print(f"{'='*50}")
print(f"Final Train Loss: {train_losses[-1]:.2f}")
print(f"Final Val Loss:   {val_losses[-1]:.2f}")
print(f"Final Test Loss:  {test_loss.item():.2f}")
print(f"R-squared:        {r_squared.item():.4f}")

print(f"\nFeature Weights:")
print("-" * 30)
for i, (name, weight) in enumerate(zip(data.feature_names, weights)):
    print(f"  {name:<10}: {weight:.4f}")


PYTORCH RESULTS
Final Train Loss: 2920.90
Final Val Loss:   3047.84
Final Test Loss:  3022.30
R-squared:        0.4296

Feature Weights:
------------------------------
  age       : 0.2646
  sex       : -11.4390
  bmi       : 26.0019
  bp        : 15.1919
  s1        : -45.8206
  s2        : 23.9474
  s3        : 8.3681
  s4        : 13.5494
  s5        : 37.1549
  s6        : 4.5241
